## Chronos
### This script should ideally be used in google colab

In [ ]:
!pip install -q "chronos-forecasting==1.5.3" "transformers==4.57.0" pandas pyarrow

import torch, transformers, importlib.metadata

print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("chronos-forecasting:", importlib.metadata.version("chronos-forecasting"))


In [ ]:
# =========================================================
# Chronos-Bolt-small target-only rolling forecast (h=1)
# - Model: amazon/chronos-bolt-small (Zero/Few-Shot quantile API)
# - Warm-up: 13 (implicit via INITIAL_WINDOW)
# - Context kept on CPU tensor to avoid device mismatch
# - Features per t: p50
#   -> Index = t; prediction is for t+1
# =========================================================

# --- Params ---
TARGET_PATH     = "target.csv"
INDEX_COL       = "date"
Y_COL           = "IP_change"
OUTPUT_PATH     = "chronos_bolt.parquet"

MODEL_ID        = "amazon/chronos-bolt-small"
USE_GPU         = True
INITIAL_WINDOW  = 6
SEED            = 42

# --- Imports & seeds ---
import numpy as np, pandas as pd, torch
from pathlib import Path
torch.manual_seed(SEED); np.random.seed(SEED)

# --- Data utils ---
def load_y(path, y_col, index_col="date"):
    path = str(path)
    if path.endswith(".csv"):
        obj = pd.read_csv(path, index_col=index_col, parse_dates=True)
    elif path.endswith(".parquet"):
        obj = pd.read_parquet(path)
        if index_col in obj.columns:
            obj[index_col] = pd.to_datetime(obj[index_col])
            obj = obj.set_index(index_col)
    else:
        raise ValueError("Use .csv or .parquet")
    s = obj[y_col] if isinstance(obj, pd.DataFrame) and y_col in obj.columns else obj.iloc[:, 0]
    if not isinstance(s.index, pd.DatetimeIndex):
        raise ValueError("Target must have a DatetimeIndex")
    return s.astype(float)

# --- Load target ---
y   = load_y(TARGET_PATH, Y_COL, INDEX_COL)
idx = y.index
m   = len(y)

# --- Chronos-Bolt pipeline (quantile API) ---
from chronos import BaseChronosPipeline
DEVICE = "cuda" if (USE_GPU and torch.cuda.is_available()) else "cpu"
print(f"[Chronos-Bolt] Using device: {DEVICE}")

pipe = BaseChronosPipeline.from_pretrained(
    MODEL_ID,
    device_map=DEVICE,          # place model on GPU/CPU
    torch_dtype=torch.float32   # official param name per README
)

# --- Rolling 1-step quantiles ---
q_levels = [0.50]

rows = []
with torch.inference_mode():
    for t in range(INITIAL_WINDOW - 1, m - 1):
        # Causal context up to (and including) t; keep as CPU tensor
        ctx_np = y.iloc[:t + 1].values.astype("float32", copy=False)
        ctx = torch.tensor(ctx_np, dtype=torch.float32)

        # predict_quantiles returns (quantiles, mean)
        quantiles, _ = pipe.predict_quantiles(
            context=ctx,
            prediction_length=1,
            quantile_levels=q_levels
        )

        # Shape: quantiles [B, H, Q]; here B=1, H=1, Q=1
        q = quantiles.detach().cpu().numpy().squeeze()

        # Robust extraction for single quantile (p50)
        if q.ndim == 0:
            p50 = float(q)
        elif q.ndim == 1:
            p50 = float(q[0])
        elif q.ndim == 2:
            p50 = float(q[0, 0])
        else:
            p50 = float(q.reshape(-1)[0])

        # Prediction is for t+1 -> timestamp = idx[t+1]
        ts = idx[t + 1]

        rows.append((ts, p50))

        if len(rows) % 50 == 0:
            print(f"[Chronos-Bolt] Step {len(rows)} @ t={t} -> {ts}")

# --- Frame & save ---
DF = pd.DataFrame(
    rows,
    columns=["date", "chronos_p50"]
).set_index("date")

DF = DF.reindex(idx).astype("float32")  # NaNs before warm-up are expected
DF.index = DF.index.strftime("%Y-%m-%d")

Path(OUTPUT_PATH).parent.mkdir(parents=True, exist_ok=True)
DF.to_parquet(OUTPUT_PATH)
print("[Chronos-Bolt] Wrote:", OUTPUT_PATH, DF.shape)
